In [1]:
from numpy import average
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, to_categorical
from sklearn.metrics import roc_auc_score
import scipy.special
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sys


from energyflow.archs.dnn import DNN
from energyflow.datasets import qg_nsubs
from energyflow.utils import data_split, to_categorical

from sklearn.metrics import roc_auc_score, roc_curve

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

2023-08-29 13:06:10.356401: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 13:06:10.388934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 13:06:10.545176: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-29 13:06:10.547057: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 13:06:11.206346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

Num GPUs Available:  0
GPUs Available:  []


/home/rikab/Documents/Research/MomentAnalysis/energyflow/archs/__init__.py:30: UserWarning: could not import some architectures - cannot import name 'cnn' from partially initialized module 'energyflow.archs' (most likely due to a circular import) (/home/rikab/Documents/Research/MomentAnalysis/energyflow/archs/__init__.py)
  warnings.warn('could not import some architectures - ' + str(e))
/home/rikab/Documents/Research/MomentAnalysis/energyflow/archs/__init__.py:40: UserWarning: could not import some architectures - cannot import name 'linear' from partially initialized module 'energyflow.archs' (most likely due to a circular import) (/home/rikab/Documents/Research/MomentAnalysis/energyflow/archs/__init__.py)
  warnings.warn('could not import some architectures - ' + str(e))


# Parameters and Data

Define the observable $\mathcal{O}^{(n)} = \sum_i z_i (c_1 + c_2 \log^n(\theta_i + c_3))$

In [2]:
# Parameters 
train = 650000
val = 50000
test = 50000
k_order = 1

epochs = 50
batch_size=512
callbacks =None
verbose = 2

# directory = f"/n/home01/rikab/moment/models/O{k_order}/"
###########

num_models_to_train = 3 ##number of models to use to make error bars
order_list = [k_order,] #
input_dim = 2
output_dim = 1


max_L = 2**(8-k_order)
F_width = 100
Phi_width = 100


def log_features(x):


    a = -3.5
    b = -1.0
    c = 0.005

    zs = x[:,0]

    mask = x[:,0] > 0
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg

    rs = np.sqrt(np.sum(np.square(x[:,1:3]), axis = 1))
    ls = a + b*np.log(rs + c)

    # ls = rs

    l_list = []

    for n in range(8 + 1):

        l_list.append(np.sum(zs * np.power(ls, n)))
    


    return l_list




# Data 
X, Y = qg_jets.load(train+val+test)

print("Data loaded!")


def cumulants(x):

    x[4,:] = np.nan_to_num(np.power(x[4,:] - 4*x[3,:]*x[1,:] - 3*x[2,:]**2 + 12*x[2,:]*x[1,:]**2 - 6*x[1,:]**4, 1/4))
    x[3,:] = np.nan_to_num(np.power(x[3,:] - 3*x[2,:]*x[1,:] + 2*x[1,:]**3, 1/3))
    x[2,:] = np.nan_to_num(np.power(x[2,:] - x[1,:]**2, 1/2))

    return x

# Data 
X, Y = qg_jets.load(train+val+test)

print("Data loaded!")


X = X[:,:,:3].astype(np.float32)
Y2 = to_categorical(Y, num_classes=2)
features = []
for x in X:
    mask = x[:,0] > 0
    yphi_avg = average(x[mask,1:3], weights=x[mask,0], axis=0)
    x[mask,1:3] -= yphi_avg
    x[mask,0] /= x[:,0].sum()

    features.append((log_features(x)))

features = cumulants(np.array(features))





Data loaded!
Data loaded!


/tmp/ipykernel_41048/2170108638.py:65: RuntimeWarning: invalid value encountered in power
  x[4,:] = np.nan_to_num(np.power(x[4,:] - 4*x[3,:]*x[1,:] - 3*x[2,:]**2 + 12*x[2,:]*x[1,:]**2 - 6*x[1,:]**4, 1/4))
/tmp/ipykernel_41048/2170108638.py:67: RuntimeWarning: invalid value encountered in power
  x[2,:] = np.nan_to_num(np.power(x[2,:] - x[1,:]**2, 1/2))


# Tensorflow fit

Fit to the function:

$F(\mathcal{O}) = \sigma(w_0 + \sum_{i=1}^k w_i(\mathcal{O}^{(i)} - c_i)^2)$

In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Model input and placeholders for w and c
input_shape = (4,)  
input_data = Input(shape=input_shape)

b1 = tf.Variable(tf.random.normal((), mean=0.0, stddev=1), trainable=True, name="bias_numerator")
b2 = tf.Variable(tf.random.normal((), mean=0.0, stddev=1), trainable=True, name="bias_denominator")

w1_i = tf.Variable(tf.random.normal((4,2), mean=0.0, stddev=1), trainable=True, name="weights_numerator")
w2_i = tf.Variable(tf.random.normal((4,2), mean=0.0, stddev=1), trainable=True, name="weights_denominator")

numerator = tf.reduce_sum(w1_i[:,0] * input_data , axis = -1) + tf.reduce_sum(w1_i[:,1] * input_data**2, axis = -1) + b1
denominator = tf.reduce_sum(w2_i[:,0] * input_data , axis = -1)  + tf.reduce_sum(w2_i[:,1] * input_data**2, axis = -1) + b2

# layer = tf.reduce_sum([layer, w_0])
output = tf.sigmoid(numerator / denominator)

# Build the model
model = Model(inputs=input_data, outputs=output)

# BCE loss
def custom_loss(y_true, y_pred):
    return tf.losses.binary_crossentropy(y_true, y_pred, from_logits=False)



In [29]:
import warnings
warnings.filterwarnings("ignore")

k_order = 4
(X_train, X_val, X_test,
    Y_train, Y_val, Y_test) = data_split(features[:,1:k_order+1], Y, val=val, test=test)

print(X_train.shape)
batch_size = 512

for epoch in range(100):

    for i in range(0, len(X_train), batch_size):
    
        x_batch = X_train[i:i+batch_size]
        y_batch = Y_train[i:i+batch_size]

        # Compute gradients with respect to w and c
        with tf.GradientTape() as tape:
            predictions = model(x_batch)
            loss = custom_loss(y_batch, predictions)

        gradients = tape.gradient(loss, [w1_i, w2_i, b1, b2])

        # Create an optimizer
        optimizer = Adam(learning_rate=0.001)

        # Apply gradients to variables
        optimizer.apply_gradients(zip(gradients, [w1_i, w2_i, b1, b2]))


    if (epoch + 1) % 1 == 0:
        y_pred = model(X_val)
        auc = roc_auc_score(Y_val, y_pred)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy()}, AUC: {auc}")


# Get the optimized values of w and c
optimized_w1 = w1_i.numpy()
optimized_w2 = w2_i.numpy()
optimized_b1 = b1.numpy()
optimized_b2 = b2.numpy()

print("Optimized w1_i:", optimized_w1)
print("Optimized w2_i:", optimized_w2)
print("Optimized b1:", optimized_b1)
print("Optimized b2:", optimized_b2)

(650000, 4)


KeyboardInterrupt: 